In [1]:
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
import pandas_gbq

In [2]:
# Assign BigQuery Credentials to Environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'crypto-valuation-490918052e31.json'

### Supplementary Data File

In [3]:
# Load Supplementary Data (Gold, SP500 & Nasdaq Indices & US 10yr Bond Rate)
df_supplementary = pd.read_csv('supplementary_financial_data.csv')
df_supplementary['Date'] = pd.to_datetime(df_supplementary['Date'], format='%Y/%m/%d')
df_supplementary.head()

,Date,Gold,SP500,Nasdaq,US_Bond
0,2018-07-02,1241.7,2726.71,7097.82,2.871
1,2018-07-03,1253.5,2713.22,7014.55,2.833
2,2018-07-05,1261.7,2736.61,7101.05,2.833
3,2018-07-06,1255.8,2759.82,7207.33,2.824
4,2018-07-09,1259.6,2784.17,7276.00,2.858


### Cryptocurrency Price Data

In [4]:
# Load the Bitcoin Cryptocurrency data
df_crypto_prices = pd.read_csv('Bitcoin Historical Data.csv', usecols=['Date','Price'])
df_crypto_prices = df_crypto_prices.rename(columns={'Price':'Bitcoin'})
df_crypto_prices['Bitcoin'] = df_crypto_prices['Bitcoin'].str.replace(',','').astype('float')

In [5]:
# Iterate through Cryptocurrency CSV files and Merge to dataframe
crypto_list = ['Ethereum','Cardano','XRP','Dogecoin']

for crypto in crypto_list:
    # Load CSV for cryptocurrency with concatenated filename from list
    file_name = crypto + " Historical Data.csv"

    temp_df = pd.read_csv(file_name, usecols = ['Date','Price'])
    
    # Remove commas from price data to convert to Float type
    if crypto in (['Ethereum','Bitcoin']): 
        temp_df['Price'] = temp_df['Price'].str.replace(',','')
    
    temp_df = temp_df.rename(columns={'Price':crypto})
    temp_df[crypto] = temp_df[crypto].astype('float')
    
    # Merge new price data onto existing dataframe with a Left join corresponding to date
    df_crypto_prices = pd.merge(df_crypto_prices, temp_df, how='left',on='Date')

In [6]:
# Conver the Date column to a datetime type
df_crypto_prices['Date'] = pd.to_datetime(df_crypto_prices['Date'], format='%m/%d/%Y')

In [7]:
# Sort into Date order and reset index to date order
df_crypto_prices = df_crypto_prices.sort_values(by='Date').reset_index()
df_crypto_prices = df_crypto_prices.drop(['index'], axis=1)
df_crypto_prices.head()

,Date,Bitcoin,Ethereum,Cardano,XRP,Dogecoin
0,2018-07-01,6366.8,452.57,0.1421,0.46226,0.002536
1,2018-07-02,6619.5,475.70,0.1544,0.49005,0.002735
2,2018-07-03,6513.5,461.89,0.1498,0.48516,0.002655
3,2018-07-04,6598.4,467.52,0.1514,0.49194,0.002688
4,2018-07-05,6546.5,468.47,0.1475,0.47779,0.002614


In [8]:
# Merge Supplementary Data with Cryptocurrency Price Data with Left join 
# from Cryptocurrency data(left) to Supplementary data (right) 
df_financial_data = pd.merge(df_crypto_prices, df_supplementary, how='left', on='Date')

# Cryptocurrency is traded every day, while the other indicies are only traded on weekdays
# Fill in the missing values with the previous days value (or the next one for the first observation)
df_financial_data = df_financial_data.fillna(method='ffill').fillna(method='bfill')

In [9]:
df_financial_data.head()

,Date,Bitcoin,Ethereum,Cardano,XRP,Dogecoin,Gold,SP500,Nasdaq,US_Bond
0,2018-07-01,6366.8,452.57,0.1421,0.46226,0.002536,1241.7,2726.71,7097.82,2.871
1,2018-07-02,6619.5,475.70,0.1544,0.49005,0.002735,1241.7,2726.71,7097.82,2.871
2,2018-07-03,6513.5,461.89,0.1498,0.48516,0.002655,1253.5,2713.22,7014.55,2.833
3,2018-07-04,6598.4,467.52,0.1514,0.49194,0.002688,1253.5,2713.22,7014.55,2.833
4,2018-07-05,6546.5,468.47,0.1475,0.47779,0.002614,1261.7,2736.61,7101.05,2.833


In [10]:
df_financial_data.to_csv('crypto_financial_data.csv', index=False)

### Bitcoin Blockchain Data

In [11]:
# Initialise and Create bigquery client object
client = bigquery.Client()

In [12]:
# Query the Bigquery Database for Daily Transaction counts and Average fee and transaction values
# between 1/7/2018 and 31/12/2023
sql_query = '''
    SELECT DATE(block_timestamp) as date_day,
       COUNT(*) as transactions_count,
       ROUND(AVG(input_value),2) as average_transaction_value,
       ROUND(AVG(fee),2) as average_fee
     FROM `bigquery-public-data.crypto_bitcoin.transactions` 
     WHERE block_timestamp_month BETWEEN "2018-07-01" AND "2023-12-31"
     GROUP BY DATE(block_timestamp)
'''
# Read into pandas dataframe with specified BigQuery Project identifier
btc_transaction_data = pandas_gbq.read_gbq(sql_query, project_id='crypto-valuation')

btc_transaction_data.head()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,date_day,transactions_count,average_transaction_value,average_fee
0,2023-12-04,570395,193371674.800000000,26340.110000000
1,2021-07-26,261950,877687564.350000000,6978.580000000
2,2021-07-12,220269,761376316.130000000,13039.090000000
3,2023-12-05,514962,190389135.430000000,45962.460000000
4,2018-10-11,262467,403836676.840000000,7016.960000000


In [13]:
# Query the Bigquery Database for Count of Daily Unique Addresses used on the Blockchain
# between 1/7/2018 and 31/12/2023
sql_query = f'''
    SELECT 
        DATE(block_timestamp) as date_day,
        COUNT(DISTINCT(ARRAY_TO_STRING(addresses," "))) AS address_count
    FROM `bigquery-public-data.crypto_bitcoin.inputs`
    WHERE DATE(block_timestamp) BETWEEN '2018-07-01' AND LAST_DAY('2023-12-01',month)
    GROUP BY DATE(block_timestamp)
    '''
# Read into pandas dataframe with specified BigQuery Project identifier
btc_address_data = pandas_gbq.read_gbq(sql_query, project_id='crypto-valuation')
btc_address_data.head()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,date_day,address_count
0,2022-09-22,545297
1,2022-07-08,620204
2,2020-03-05,530249
3,2020-11-17,706559
4,2022-03-23,614284


In [22]:
# Merge the Bitcoin ledger dataframes on the date
df_btc_statistics = pd.merge(btc_transaction_data, btc_address_data, how='left', on='date_day')
df_btc_statistics['date_day'] = pd.to_datetime(df_btc_statistics['date_day'], format='%Y/%m/%d')
df_btc_statistics.head()

,date_day,transactions_count,average_transaction_value,average_fee,address_count
0,2023-12-04,570395,193371674.800000000,26340.110000000,664523
1,2021-07-26,261950,877687564.350000000,6978.580000000,603155
2,2021-07-12,220269,761376316.130000000,13039.090000000,525192
3,2023-12-05,514962,190389135.430000000,45962.460000000,522848
4,2018-10-11,262467,403836676.840000000,7016.960000000,484680


In [23]:
df_btcreward = pd.read_csv('BitcoinReward.csv')
df_btcreward['DateTime'] = pd.to_datetime(df_btcreward['DateTime'].str[:-5], format='%d/%m/%Y')
df_btcreward = df_btcreward.rename(columns={'DateTime':'Date'})

In [26]:
df_btc_statistics = pd.merge(df_btc_statistics, df_btcreward, how='left',left_on='date_day', right_on='Date')
df_btc_statistics = df_btc_statistics.drop(['Date'], axis=1)

In [27]:
df_btc_statistics.head()

,date_day,transactions_count,average_transaction_value,average_fee,address_count,Block Reward
0,2023-12-04,570395,193371674.800000000,26340.110000000,664523,6.25
1,2021-07-26,261950,877687564.350000000,6978.580000000,603155,6.25
2,2021-07-12,220269,761376316.130000000,13039.090000000,525192,6.25
3,2023-12-05,514962,190389135.430000000,45962.460000000,522848,6.25
4,2018-10-11,262467,403836676.840000000,7016.960000000,484680,12.50


In [28]:
df_btc_statistics.to_csv('bitcoin_statistics.csv', index=False)